# Synthetic Ramesside Star Clocks

In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import os 
import pandas as pd
import matplotlib.pyplot as plt
import math
import star_chart_spherical_projection as scsp
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun, Angle, Longitude
from sunpy.coordinates import frames, sun

# I want get_sun to shut up so ignoring warnings:
import warnings
warnings.simplefilter('ignore', UserWarning)

## Functions

In [6]:
def StarRiseSet(jd, starAlt, deg):
    # create return arrays
    star_rise = np.zeros(int(len(jd)/360), dtype=int)
    star_set = np.zeros(int(len(jd)/360), dtype=int)
    # loop through each day
    for i in range(360, len(jd) + 1, 360):
        # isolate data for one day
        dailyStarAlt = starAlt[i - 360 : i]
        # is the star visible at the beginning of the day
        starVis = dailyStarAlt[0] > deg
        # make array of booleans
        bool_arr = dailyStarAlt > deg   
        # find two indices of change
        ind1 = np.argwhere(bool_arr != starVis)[0][0]
        if len(np.argwhere(bool_arr[ind1:]== starVis)) != 0: # edge case of next index is in next day
            ind2 = np.argwhere(bool_arr[ind1:]== starVis)[0][0] + ind1
        else:
            ind2 = 360    
        # assign indices
        if starVis:
            star_set[int((i-360)/360)] = int(-360 + i  + ind1)
            star_rise[int((i-360)/360)] = int(-360 + i  + ind2)
        else:
            star_set[int((i-360)/360)] = int(-360 + i  + ind2)
            star_rise[int((i-360)/360)] = int(-360 + i  + ind1)
    return (star_rise, star_set)

def SunRiseSet(jd, SunAlt, deg):
    
    '''
    A function to create a list of indices where the Sun rises and sets in a given year. 
    This is useful for making sure we're tracking nightly, visible motion of the decans.
    NOTE: as written, this code assumes that data is collected every 4 minutes. 
    To change this, change number to number of collection intervals per day! (360 = 24 * 60/4)
    Inputs: 
        jd = Julian date
        SunAlt = the altitude of the Sun
    Outputs:
        sunriseset = indices of sunrise and sunset in the jd & date columns
    '''
    
    sunriseset = []
    for i in range(360, len(jd), 360):
        temp = []
        for j in range(i - 360, i):
            if SunAlt[j] <= deg + 0.4 and SunAlt[j] >= deg - 0.4:
                if len(temp) == 0: 
                    temp.append(j)
                elif temp[-1] != j - 1:
                    temp.append(j)
        sunriseset.append(temp)
    return sunriseset


def isStarVisible(sunSet, sunRise, starAlt):
    vis_arr = np.full(365, True)
    max_alt_arr = np.zeros(364)
    for i in range(0, 364):
        maxalt = max(starAlt[sunSet[i]:sunRise[i + 1]])
        max_alt_arr[i] = maxalt
        #print(maxalt)
        if maxalt < 0:
            vis_arr[i] = False
    return(max_alt_arr, vis_arr)


def MaxMinAltAz(jd, sunriseset, DecAz, DecAlt):
    
    '''
    A function to create lists of minimum and maximum azimuths and altitudes of the decan. 
    This is useful for making sure we're tracking nightly, visible motion of the decans.
    Inputs: 
        direct = string with the directory where the .txt file is located
        filename = string with name of file (name + month + year)
        jv = Julian date
        sunriseset = indices of sunrize and sunset in the jd & date columns
    Outputs:
        sunriseset = indices of sunrize and sunset in the jd & date columns
        days = list of indices when it's daylight 
        minaz, maxaz = minimum and maximum azimuths of the decan per night
        minalt, maxalt = minimum and maximum altitudes of the decan per night
        riseaz, setaz = azimuth of decan at rise & set
        risealt, setalt = altitude of decan at rise & set
    '''
    maxalt = []
    minalt = []
    maxaz = []
    minaz = []
    riseaz = []
    setaz = []
    risealt = []
    setalt = []
    days = []
    for i in range(0, int(len(jd)/360) - 1):
        sset = sunriseset[i][1]
        srise = sunriseset[i + 1][0]
        maxalt.append(max(DecAlt[sset:srise]))
        minalt.append(min(DecAlt[sset:srise]))
        maxaz.append(max(DecAz[sset:srise]))
        minaz.append(min(DecAz[sset:srise]))
        riseaz.append(DecAz[srise])
        setaz.append(DecAz[sset])
        risealt.append(DecAlt[srise])
        setalt.append(DecAlt[sset])
        days.append(DecAlt[srise:sset])
    return(days, minaz, maxaz, minalt, maxalt, riseaz, setaz, risealt, setalt)

def horizonBins(horizon, bsize, gsize):
    # if gap size is zero, return as usual
    if gsize == 0:
        return((np.linspace(horizon[0], horizon[1], 8), 1))
    # otherwise, divide into smallest common denominator
    num = 7 * bsize + 7 * gsize
    horizon_bins = np.linspace(horizon[0], horizon[1], num)
    # select indices to merge by bin/gap size
    ## first gaps
    ind_gap = np.zeros(14)
    ind_gap[1::2] = np.arange(0,7)
    ind_gap[::2] = np.arange(0,7)
    ## then bins
    ind_bin = np.zeros(14)
    ind_bin[0:13] = ind_gap[1:]
    ind_bin[-1] = 7
    # add together with bin/gap sizes
    inds = ind_bin * bsize + ind_gap * gsize
    inds = inds.astype(int)
    # return only those bin and gap indices
    return((horizon_bins[inds], 2))

# def synRSC1star(date_ind, horizon, bsize, gsize, sunSet, sunRise, starAz, starVis):
#     '''Function which, when given a date, size of horizon, and star data*, 
#     makes a synthetic Ramesside Star Clock.'''
#     # define horizon bin limits (assuming equal binsize)
#     (horizon_bins, skip) = horizonBins(horizon, bsize, gsize)
#     # define time bin limits (assuming equal binsize)
#     sset = sunSet[date_ind]
#     sris = sunRise[date_ind + 1]
#     #time_bins = np.linspace(jd[sset], jd[srise], 14)
#     time_ind_bins = np.round(np.linspace(sset, sris, 13))
#     # bin star as appropriate 
#     rsc_table = []
#     for i in range(0, 13):
#         ind = int(time_ind_bins[i])
#         if starVis[date_ind]:
#             az_list = starAz[ind]
#             row = np.histogram(az_list, horizon_bins)[0][0:-1:skip]
#             #print("hour " + "{:02d}".format(i), row)
#         else:
#             row = np.zeros(7)       
#         rsc_table.append(row)
#     return(np.array(rsc_table))

def isStarInWindow(alt_window, starAlt):
    '''
    Function to check whether star is in a given altitude window.
    '''
    (alt_min, alt_max) = alt_window
    if starAlt >= alt_min and starAlt <=alt_max:
        return(True)
    else:
        return(False)


def synRSC1star(date_ind, alt_window, horizon, bsize, gsize, sunSet, sunRise, starName, starAz, starAlt, starVis):
    '''Function which, when given a date, size of horizon, and single star's data*, 
    makes a synthetic Ramesside Star Clock.'''
    # define horizon bin limits (assuming equal binsize)
    (horizon_bins, skip) = horizonBins(horizon, bsize, gsize)
    # define time bin limits (assuming equal binsize)
    sset = sunSet[date_ind]
    sris = sunRise[date_ind + 1]
    #time_bins = np.linspace(jd[sset], jd[srise], 14)
    time_ind_bins = np.round(np.linspace(sset, sris, 13))
    # bin star as appropriate 
    rsc_table = []
    for i in range(0, 13):
        ind = int(time_ind_bins[i]) # is this the index I want to be passing? 
        inWindow = isStarInWindow(alt_window, starAlt[ind])
        if starVis[date_ind] and inWindow:
            az_list = starAz[ind]
            row = np.histogram(az_list, horizon_bins)[0][::skip]
            #print(date_ind)
            #print("in Window=" + str(inWindow)+", az_list = "+ str(az_list))
        else:
            row = np.zeros(7, dtype=int)  
        rsc_table.append(row)
    return(np.array(rsc_table))

def synRSC(date_ind, alt_window, horizon, bsize, gsize, sunSet, sunRise, starlist, starAzlist, starAltlist, starVislist):
    '''
    Function to create synthetic Ramesside Star Clocks given data from several stars.
    '''
    # how many stars
    df = pd.DataFrame(data=np.empty((13,7), dtype=str))
    # iterate over stars
    for i in range(len(starlist)):
        temp_table = synRSC1star(date_ind, alt_window, horizon, bsize, gsize, sunSet, sunRise, starlist[i], starAzlist[i], starAltlist[i], starVislist[i])
        inds = np.argwhere(temp_table == 1)
        for ind in inds:
            df.at[ind[0], ind[1]] += (starlist[i] + " ")   
    df.columns = [-3, -2, -1, 0, 1, 2, 3]
    return df

def mag_data(df, mag_dict):
    '''
    Given a data frame made with synRSC and a name-to-magnitude value dictionary, 
    this function will select the brightest star in each row (aka horizon bin) to 
    create a magnitude-selected Ramesside Star Clock.  
    '''
    # data frame to save magnitude-selected data
    df_mag = pd.DataFrame(data=np.empty((13,7), dtype=str))
    # iterate through df of all possible stars and select for magnitude
    for i in range(0, 13):
        sname = ""
        min_mag = 10 # all human visible magnitudes should be higher than this 
        # iterate through columns in row ( = horizon bins)
        for j in range(-3, 4):
            dlist = list(filter(None, df[j][i].split(' '))) # split into star names and filter out empty strings 
            for item in dlist:
                if mag_dict[item] < min_mag:
                    min_mag = mag_dict[item] # update brightest available star
                    cind = j + 3 # column index
                    sname = item # star name 
        if len(sname) > 1:                     
            df_mag.at[i, cind] = sname
    df_mag.columns = [-3, -2, -1, 0, 1, 2, 3]
    return(df_mag)


## Import Data & Make Necessary Structures

In [3]:
### Set the directory
direct = os.getcwd() # current working directory
direct = direct + '/DecanLists/' # directory where the .txt files go
filename = "mockdata_200_1300BC-Jan-29-2024_1422.txt" # which data file (generated by mockrun.py)

In [7]:
decanOutput = pd.read_csv(direct + filename, sep = "|")

# get header 
header = decanOutput.keys()

# get list of star names
starlist = []
for star_name in header[4:-1:2]:
    starlist.append(star_name[0:-8])

# get standard data
jd = decanOutput[header[0]].to_numpy()
hrd = decanOutput[header[1]]
sunAz = decanOutput[header[2]].to_numpy()
sunAlt = decanOutput[header[3]].to_numpy()

# get star data
num_decs = int( (len(header) - 4) / 2 ) # how many stars in list 
starsAz = np.zeros((num_decs, len(jd)))
starsAlt = np.zeros((num_decs, len(jd)))
for i in range(0, num_decs):
    starsAz[i, :] = decanOutput[header[4 + 2 * i]].to_numpy()
    starsAlt[i, :] = decanOutput[header[5 + 2 * i]].to_numpy()
   
# get sunrise and sunset times
(sunRise, sunSet) = StarRiseSet(jd, sunAlt, -12)
sunAzSet = sunAz[sunSet]

# get star rise and set times
rise_alt_deg = 10 # define starrise in muber of degrees above horizon 
starAzRiseList = np.zeros((num_decs, len(sunRise)))
starVisList = np.full((num_decs, len(sunRise)), True)
starMaxAltList = np.zeros((num_decs, len(sunRise)-1))
# loop over all decans
for i in range(0, num_decs):
    # create visibility and maximum altitude array
    if np.min(starsAlt[i]) >= rise_alt_deg: 
        # if the star is circumpolar at the given location
        starVisList[i,:] = np.full((365,), True)
        starMaxAltList[i,:] = np.max(starsAlt[i])
    elif np.max(starsAlt[i]) < rise_alt_deg:
        # if a star never rises above rise_alt_deg
        starVisList[i,:] = np.full((365,), False)
        starMaxAltList[i,:] = np.max(starsAlt[i])
    else: 
        # otherwise, check when the star isn't visible
        (starRise, starSet) = StarRiseSet(jd, starsAlt[i], rise_alt_deg)    #???
        starAzRise = starsAz[i, starRise]                                   #???
        starAzRiseList[i, :] = starAzRise                                   #???
        (maxAlt, starVis) = isStarVisible(sunSet, sunRise, starsAlt[i])
        starVisList[i,:] = starVis
        starMaxAltList[i,:] = maxAlt


## Create Synthetic RSCs

In [12]:
alt_window = (0, 45) # degrees 
horizon = (160, 200) # Note: MUST have smaller number first; 
bsize = 1 # bin size (must be 1 if gsize = 0)
gsize = 0 # gap size (relative to binsize) 

# make dictionary of names and magnitudes (if not already in memory) 
# THIS NEEDS TO BE GENERALIZED
# I should really rethink my folder structure
name_df = pd.read_csv('/Users/lunazagor/Code/GitHub/decanOpy/DecanLists/ICs/star_data_Jan-29-2024_1422.csv', index_col=None, header=0, names=['Name', 'RA', 'Dec', 'Mag'])
mag_dict = name_df.set_index('Name')['Mag'].to_dict()

# Creating Excel Writer Object from Pandas  
folder = "./SynRSC/"
wname = 'synRSC_FS_hor' + str(horizon[0]) + '-' + str(horizon[1]) + '_b=' + str(bsize) + '_g=' + str(gsize) + '_' + str(num_decs) + '.xlsx'

writer = pd.ExcelWriter(folder + wname, engine='xlsxwriter')     
workbook=writer.book
worksheet=workbook.add_worksheet('RSCs')
writer.sheets['RSCs'] = worksheet
worksheet2=workbook.add_worksheet('Mag Select')
writer.sheets['Mag Select'] = worksheet2
# Format test
format = workbook.add_format()
format.set_font_size(11)
# Write down some important data
worksheet.write(0, 0, "horizon is " + str(horizon), format)
worksheet.write(1, 0, "alt window is " + str(alt_window), format)
worksheet.write(2, 0, "bsize = " + str(bsize), format)
worksheet.write(3, 0, "gsize = " + str(gsize), format)

# Write data (saved in /SynRsc folder)

for i in range(0, 24):
    date = i * 15 # days from first day in decan data
    df = synRSC(date, alt_window, horizon, bsize, gsize, sunSet, sunRise, starlist, starsAz, starsAlt, starVisList)
    #df_test.to_excel('output' + str(date) + '.xlsx', index=False)
    df.to_excel(writer, sheet_name='RSCs',startrow= i * 15 + 5, startcol=0)   
    worksheet.write(i * 15 + 5,  0, "Table " + str(i + 1), format)
    #another_df.to_excel(writer,sheet_name='RSCs',startrow=20, startcol=0) 
    #attempt to add mag data
    df_mag = mag_data(df, mag_dict)
    df_mag.to_excel(writer, sheet_name='Mag Select',startrow= i * 15 + 5, startcol=0) 
    worksheet2.write(i * 15 + 5,  0, "Table " + str(i + 1), format)
writer.close()    

Ha, it works! Some notes on things to do:

- general filestructure cleanup
    - the dictionary importing is hard coded right now--have it depends on direct, folder, filename, etc 
    - speaking of which, let's start naming files and folders similarly, so that IC filename --> decanOpy output folder --> SynRSC folder 
    - Clean up extraneous files and backup on git!
- maybe separate IC generation from decanOpy run?
    - this would involve figuring out how to point to a different star data csv for the real sky, which I have to do anyway
- RSC generator should be wrapped up in at least one more layer of functions
    - functions can also be stored in separate file 
    - in fact, decanO.py and co should probably go in their own folder and be imported properly (ayayay)
    - go through and comment the functions
    - generate readthedocs!
- RSC generator should have more choices
    - magnitude
    - "known star" THEN magnitude 
    - closest to center of bin 


    - Fake stars
        - maximum magnitude 4 
        - make sure not too many bright stars -- maybe not a uniform randomness?


        Largest Hipaprchos number for real data is 116727:
         -  name data as letter + 6 numbers, always a string 7-elements long


In [107]:
# inv_mag_dict = dict((v, k) for k, v in mag_dict.items())  ### invert dictionary

# # Python code to demonstrate 
# # finding duplicate values from a dictionary

# # initialising dictionary
# ini_dict = mag_dict


# # printing initial_dictionary
# print("initial_dictionary", str(ini_dict))

# # finding duplicate values
# # from dictionary
# # using a naive approach
# rev_dict = {}

# for key, value in ini_dict.items():
# 	rev_dict.setdefault(value, set()).add(key)
	
# result = [key for key, values in rev_dict.items()
# 							if len(values) > 1]

# # printing result
# print("duplicate values", str(result))
